In [1]:
import schedule # conda install -c conda-forge schedule로 인스톨 한 버전(1.2.2)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import date
import pandas as pd
from datetime import datetime, timedelta

flag=0
def crawl_job():
    
    global flag
    
    print("웹크롤링 실행 중...")
    driver = webdriver.Chrome()
    driver.get("https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101")
    time.sleep(4) # 데이터 로딩시간의 문제로 추가
    
    # 시작일(11월 28일(금)[12월 1일의 증감치 확인을 위해])
    start_day = pd.to_datetime('20251128', format='%Y%m%d')
    today = datetime.today()
    date_dff = (today-start_day).days
    chg_date = start_day
    
    date_input = driver.find_element(By.ID, "trdDd")
    driver.execute_script("arguments[0].value = arguments[1];", date_input, start_day.strftime("%Y%m%d"))

    search_btn = driver.find_element(By.ID, "jsSearchButton") 
    search_btn.click()
    time.sleep(4)
    
    krx_name_list =[]
    closing_price_list =[]
    trade_value_list = []
    krx_list =[]
    result_list =[]
    
    for date_idx in range(date_dff+1):

        krx_name = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-LEFT[data-name="IDX_NM"]')
        closing_price = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="CLSPRC_IDX"]')
        trade_value_list = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="ACC_TRDVOL"]')

        krx_name_list = [items.text for items in krx_name if items.text != ""]
        closing_price_list = [items.text.replace(",","") for items in closing_price if items.text != ""]
        trade_value_list = [items.text.replace(",","") for items in trade_value_list if items.text != ""]
        date_maker = chg_date.strftime("%Y%m%d")

        for idx, item, in enumerate(krx_name_list):
            krx_list.append([date_maker, krx_name_list[idx], closing_price_list[idx], trade_value_list[idx]])
        keywords = ['건설', '헬스케어', '자동차'] 
        fdate = driver.find_element(By.CSS_SELECTOR, 'input[name="trdDd"]')

        result_list = [ [date, name, price, trade] for date, name, price, trade in krx_list if any(kw in name for kw in keywords) if (price != '-') or (trade != '-')] 

        chg_date += timedelta(days=1)
        date_input = driver.find_element(By.ID, "trdDd")
        driver.execute_script("arguments[0].value = arguments[1];", date_input, chg_date.strftime("%Y%m%d"))

        search_btn = driver.find_element(By.ID, "jsSearchButton") 
        search_btn.click()

        time.sleep(4)
    
    print("증감치 계산중...")
    data = pd.DataFrame(result_list, columns=['날짜', '종목명', '종가(백만원)', '거래량(천주)'])

    data.sort_values(by=['날짜','종목명'],ascending=[True,True], inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    check_data = data.iloc[:,-2].copy()
    KRX_items = set(data.iloc[:,1])
    
    data_count = len(check_data)
    KRX_count = len(KRX_items)
    date_count = int(data_count/KRX_count)
    
    up_down_list = []
    # 전체데이터에 날짜당 아이템 갯수를 나누어 묶음들을 구함
    for date_idx in range(date_count):
        # 묶음 내부의 값들의 증감치 추가
        # 첫 묶음은 비교대상이 없기에 '시작일'로 지정
        if date_idx == 0:
            for start_idx in range(KRX_count):
                up_down_list.append('시작일')
        else:
            # 현재 묶음과 이전 묶음의 각각의 아이템의 크기를 비교 
            for udw_idx in range(KRX_count):
                if check_data[date_idx*3 + udw_idx] > check_data[(date_idx-1)*3 + udw_idx]:
                    up_down_list.append('증가')
                elif check_data[date_idx*3 + udw_idx] < check_data[(date_idx-1)*3 + udw_idx]:
                    up_down_list.append('감소')
                else:
                    up_down_list.append('변동없음')
                    
    data['전일대비'] = up_down_list
    data.to_csv('data/주가지수크롤링.csv')
    print('작업완료')
    flag = 1

# 실행 시 다음 오후 4시에 실행(크롤링 사이트에 정보지연 20분 표기가 있어 여유로운 시간 지정)
schedule.every().day.at("16:00").do(crawl_job)

while flag==0:
    schedule.run_pending()
    time.sleep(60)

웹크롤링 실행 중...
증감치 계산중...
작업완료
